# AGLAE PIXE data treatment

In [ ]:
%pip install ipywidgets

In [ ]:
import bae
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import io
import pandas as pd
from pandas.api.typing import DataFrameGroupBy

In [ ]:
uploader = widgets.FileUpload()
display(uploader)

In [ ]:
pd.set_option("display.float_format", "{:.0f}".format)
def pager(df, start, nb):
    return df[start:start+nb]

In [ ]:
print(uploader.value)
pige_corrected_compositions_table = pd.read_excel(io.BytesIO(uploader.value[0].content), index_col=[0, 1, 2, 3])
normalized_compositions_table = pige_corrected_compositions_table.apply(bae.normalize, axis=1)

In [ ]:
def extract_simplified_name(row):
    name = row[2]
    if "_" in name:
        split = name.rsplit('_', 1)
        return split[0] if "pt" in split[1] or "map" in split[1] else name
    else:
        return name

x_widget = widgets.FloatSlider(min=0.2, max=3.0, step=0.1)
@interact(df=fixed(normalized_compositions_table), tolerance=x_widget, threshold=fixed(10000))
def remove_outliners(df: pd.DataFrame, tolerance, threshold=10000):
    to_delete=[]
    major_group = df[normalized_compositions_table > threshold].groupby(extract_simplified_name, sort=False)
    for name, grp in major_group:
        if grp.count().max() <= 1:
            continue
        median = grp.median()
        d = (grp - median)/median
        for i, row in d.iterrows():
            #print(row)
            for j, elt in row[row>tolerance].items():
                print(i[2], j)
                to_delete.append(i)
                #to_delete.loc[i] = j
    return to_delete

#normalized_compositions_table.loc[to_delete[0]]
#df_group.filter(detect_outliers)
#pager(res,5,10)
#pager(comp_table_without_outliners,90,10)
#for idx in to_delete:
#    print(idx[2])


In [ ]:
table_without_outliners = normalized_compositions_table.drop(remove_outliners(normalized_compositions_table, tolerance=0.5))


In [ ]:
final_table_ppm = table_without_outliners.groupby(extract_simplified_name, sort=False).agg(["mean", "std"])
pager(table_without_outliners.groupby(extract_simplified_name, sort=False).agg(["mean", "std"]), 5, 40)

In [ ]:
major_elements_list = ["Na2O PIGE", "MgO", "Al2O3", "SiO2", "P2O5", "SO3", "Cl", "K2O", "CaO", "TiO2","V2O3", "Cr2O3", "MnO", "Fe2O3", "CoO", "NiO", "CuO", "ZnO", "As2O5", "SrO", "Ag2O", "SnO2", "Sb2O5", "BaO", "Au2O3", "PbO"]

In [ ]:
final_table_ppm_major_elements = final_table_ppm[major_elements_list]
pager(final_table_ppm_major_elements,5,40)

In [ ]:
final_table_wt = (final_table_ppm / 10000).round(3)
pager(final_table_wt,5,40)

In [ ]:
final_table_wt_major_elts = final_table_wt[major_elements_list]

In [ ]:
with pd.ExcelWriter("final.xlsx") as writer:
    final_table_ppm.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm all elements")
    final_table_ppm_major_elements.map(lambda x: f"{x:.0f}").to_excel(writer, sheet_name="ppm major elements")
    final_table_wt.to_excel(writer, sheet_name="wt all elements")
    final_table_wt_major_elts.to_excel(writer, sheet_name="wt major elements")